<a href="https://colab.research.google.com/github/leaBroe/BERT_for_antibody_pairing/blob/master/protbert_nsp_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSP and MLM tasks using ProtBERT bfd
-> https://huggingface.co/Rostlab/prot_bert_bfd  
Input data has to be of the form heavyseq[SEP]lightseq with each AA single space separated!!  
example:  
Q V Q L Q E S G P G L V K P S E T L S L T C T V S G G S I S G F Y W S W I R Q S P G K G L E [SEP] Q V Q L Q E S G P G L V K P S E T L S L T C T V S G G S I S G F Y W S W I R Q S P G K G L E

In [1]:
!pip install virtualenv
!virtualenv nsp
!source nsp/bin/activate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 53.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 404ms
  creator CPython3Posix(dest=/content/nsp, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
!pip install evaluate
!pip install seqeval
!pip install wandb
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [3]:
!pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [4]:
import sys
import torch
from torch.utils.data import DataLoader
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from transformers import BertConfig, BertForPreTraining, BertLMHeadModel, BertTokenizer, logging
from transformers.data.datasets.language_modeling import TextDatasetForNextSentencePrediction
from transformers import pipeline, Trainer, TrainingArguments
from transformers import AutoConfig, AutoModel
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer
import evaluate
# from nlp import load_dataset
# from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import seqeval
import json
import math
import logging
from datetime import datetime
import pytz     # for time zone
from torch.optim import AdamW
from transformers import get_scheduler
import wandb

In [5]:
# print used device cpu or cuda/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

device: cuda


In [6]:
# Load the tokenizer and configuration
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )
config = AutoConfig.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False, vocab_size=len(tokenizer), force_download=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

In [7]:
# print tokenizer and config
print(f'tokenizer: {tokenizer}')
print(f'config: {config}')


tokenizer: BertTokenizerFast(name_or_path='Rostlab/prot_bert_bfd', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
config: BertConfig {
  "_name_or_path": "Rostlab/prot_bert_bfd",
  "architectures": [
    "Bert

In [8]:
PST = pytz.timezone('Europe/Zurich')

#instantiate the model
print("start loading model=",datetime.now(PST))
model = BertForPreTraining(config=config)

# Move model to the appropriate device (GPU or CPU)
model.to(device)  # 'device' is determined by torch.device("cuda" if torch.cuda.is_available() else "cpu")

# confirm model is on the right device
print(f"Model is on device: {next(model.parameters()).device}")  # It will show cuda:0 if on GPU

# Load your model's configuration and check the vocab_size parameter. It must match the total number of tokens in your tokenizer’s vocabulary!
print(f"Model's vocab_size: {model.config.vocab_size}")
print(f"Tokenizer's vocab_size: {tokenizer.vocab_size}")

print("Model's vocab size from embeddings:", model.bert.embeddings.word_embeddings.num_embeddings)


start loading model= 2024-06-21 14:06:11.273227+02:00
Model is on device: cuda:0
Model's vocab_size: 30
Tokenizer's vocab_size: 30
Model's vocab size from embeddings: 30


In [9]:
# Initialize wandb
run_name = "small_data_50_epochs_new_metrics_lr2e-6_colab"

wandb.init(project="paired_model_nsp_mlm_protbert", name=run_name)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
output_dir = f"./{run_name}"
logging_dir = f"./{run_name}_logging"


# define the arguments for the trainer
training_args = TrainingArguments(
    output_dir=output_dir,          # output directory
    num_train_epochs=50,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training (try 16 if needed)
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=logging_dir,     # directory for storing logs
    do_train=True,
    eval_strategy="epoch",
    logging_steps=5
)


In [11]:
from google.colab import files
uploaded = files.upload()

Saving paired_full_seqs_sep_train_no_ids_small_SPACE_separated.txt to paired_full_seqs_sep_train_no_ids_small_SPACE_separated.txt
Saving paired_full_seqs_sep_val_no_ids_small_SPACE_separated.txt to paired_full_seqs_sep_val_no_ids_small_SPACE_separated.txt


In [12]:
# small dataset with input heavyseq[SEP]lightseq with each AA SPACE SEPARATED!!
small_train_dataset_path = "/content/paired_full_seqs_sep_train_no_ids_small_SPACE_separated.txt"
small_val_dataset_path = "/content/paired_full_seqs_sep_val_no_ids_small_SPACE_separated.txt"


In [13]:
# Prepare the train_dataset
print("start building train_dataset=", datetime.now(PST))
train_dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path=small_train_dataset_path,
    block_size=512
)

# Prepare the eval_dataset
print("start building eval_dataset=", datetime.now(PST))
eval_dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path=small_val_dataset_path,
    block_size=512
)

start building train_dataset= 2024-06-21 14:07:09.957586+02:00
start building eval_dataset= 2024-06-21 14:07:10.102528+02:00


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:360: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:
# Initialize the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [15]:
# Function to create a DataLoader with debugging -> to see the first batch before and after data collator, see if it handles the data correctly
def create_data_loader_with_debugging(dataset, batch_size, data_collator):
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    for batch in data_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        next_sentence_label = batch['next_sentence_label']

        # Print the first batch to debug
        print("First batch before data collator:")
        for i, example in enumerate(dataset):
            if i == batch_size:
                break
            print(f"Example {i}: {tokenizer.decode(example['input_ids'])}")

        print("First batch after data collator:")
        print(f"Input IDs: {input_ids}")
        print(f"Attention Mask: {attention_mask}")
        print(f"Labels: {labels}")
        print(f"Next Sentence Labels: {next_sentence_label}")
        break
    return data_loader

In [16]:
# Create and debug the DataLoader
train_data_loader = create_data_loader_with_debugging(train_dataset, batch_size=16, data_collator=data_collator)


First batch before data collator:
Example 0: [CLS] Q V Q L Q E S G P G L V K P S E T L S L T C T V S G G S I S G F Y W S W I R Q S P G K G L E W I A Y I Y F S G S T N Y N P S L K S R V T L S V D T S K N Q F S L K L S S V T A A D S A V Y Y C A R D V G P Y N S I S P G R Y Y F D Y W G P G T L V T V S S [SEP] Q S A L T Q P A S V S G S P G Q S I T I S C T G T S S D V G N Y N L V S W Y Q H H P G K A P K L M I Y E V S K R P S G I S N R F S G S K S G N T A S L T I S G L Q A D D E A D Y Y C C S Y A G S R I L Y V F G S G T K V T V L Q V Q L Q E S G P G L V K P S E T L S L T C T V S G G S I S S Y H W S W I R Q P P G K G L E W I G Y M Y Y S G S T N Y N P S L K S R V T I S V D T S K T Q F S L K L S S V T T A D T A V Y Y C A R G R L I W S A D Y T G G D Y F D P W G Q G I L V T V S S [SEP] Q S A L T Q P A S V S G S P G Q S I T I S C T G S S S D V G S Y N L V S W Y Q Q H P G K A P K L M I Y E V S K R P S G V S N R F S G S K S G N T A S L T I S G L Q A E D E A Q Y Y C C S Y G G R N F H V L F G G G T E L

In [17]:
def compute_metrics(mlm_preds, mlm_labels, nsp_preds, nsp_labels):
    true_mlm_labels = []
    true_mlm_preds = []
    true_nsp_labels = nsp_labels
    true_nsp_preds = nsp_preds

    for i in range(len(mlm_labels)):
        true_mlm_labels.extend(mlm_labels[i])
        true_mlm_preds.extend(mlm_preds[i])

    # for i in range(len(nsp_labels)):
    #     true_nsp_labels.extend(nsp_labels[i])
    #     true_nsp_preds.extend(nsp_preds[i])

    print(f"length of true mlm labels {len(true_mlm_labels)}")
    print(f"length of mlm preds {len(true_mlm_preds)}")

    print(f"length of true nsp labels {len(true_nsp_labels)}")
    print(f"length of nsp preds {len(true_nsp_preds)}")

    print(f"true mlm labels {true_mlm_labels}")
    print(f"mlm predictions {true_mlm_preds}")

    print(f"true nsp labels {true_nsp_labels}")
    print(f"nsp predictions {true_nsp_preds}")

    # Filter out -100 labels (which are ignored in the evaluation)
    filtered_mlm_labels = [label for label in true_mlm_labels if label != -100]
    filtered_mlm_preds = [pred for label, pred in zip(true_mlm_labels, true_mlm_preds) if label != -100]

    print(f"length of filtered mlm labels {len(filtered_mlm_labels)}")
    print(f"length of filtered mlm preds {len(filtered_mlm_preds)}")

    print(f"Filtered mlm labels: {filtered_mlm_labels}")
    print(f"Filtered mlm preds: {filtered_mlm_preds}")

    mlm_precision, mlm_recall, mlm_f1, _ = precision_recall_fscore_support(filtered_mlm_labels, filtered_mlm_preds, average='macro', zero_division=0)
    acc_mlm = accuracy_score(filtered_mlm_labels, filtered_mlm_preds)

    nsp_precision, nsp_recall, nsp_f1, _ = precision_recall_fscore_support(true_nsp_labels, true_nsp_preds, average='macro', zero_division=0)
    acc_nsp = accuracy_score(true_nsp_labels, true_nsp_preds)

    return {
        'mlm accuracy': acc_mlm,
        'mlm f1': mlm_f1,
        'mlm precision': mlm_precision,
        'mlm recall': mlm_recall,
        'nsp accuracy': acc_nsp,
        'nsp precision': nsp_precision,
        'nsp f1': nsp_f1,
        'nsp recall': nsp_recall
    }


In [18]:
# Instantiate the trainer
print("start building trainer=",datetime.now(PST))


# Instantiate the custom trainer
class CustomTrainer(Trainer):
    def log(self, logs: dict):
        super().log(logs)
        if 'loss' in logs:
            print(f"Training loss: {logs['loss']}")

    def evaluate_and_log(self):
        metrics = self.evaluate()
        print(f"Evaluation metrics: {metrics}")
        return metrics

    def train(self, *args, **kwargs):
        for epoch in range(int(self.args.num_train_epochs)):
            # Call the parent's train method for each epoch
            super().train(*args, **kwargs)
            # Evaluate and log after each epoch
            self.evaluate_and_log()


start building trainer= 2024-06-21 14:07:32.880620+02:00


In [19]:
trainer = CustomTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,           # evaluation dataset
    compute_metrics=compute_metrics,  # pass the compute_metrics function
)


In [20]:
learning_rate = 2e-6

optimizer = AdamW(model.parameters(), lr=learning_rate)

# Create DataLoader for debugging
train_data_loader = DataLoader(train_dataset, batch_size=16, collate_fn=data_collator)
eval_data_loader = DataLoader(eval_dataset, batch_size=16, collate_fn=data_collator)


In [22]:
# Prepare the learning rate scheduler
num_training_steps = len(train_data_loader) * training_args.num_train_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=num_training_steps
)

In [24]:
import os

In [26]:
# Custom training loop with gradient clipping and detailed logging
print("Starting training...")

################# Custom Training Loop #################

for epoch in range(training_args.num_train_epochs):
    model.train()
    train_loss = 0

    for step, batch in enumerate(train_data_loader):
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Extract the loss and logits from the model output
        loss = outputs.loss
        train_loss += loss.item()
        #loss, prediction_logits, seq_relationship_logits = outputs

        # Perform backpropagation
        loss.backward()

        # Clip gradients to prevent explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update parameters and learning rate
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        print(f"Epoch: {epoch}, Step: {step}, Training Loss: {loss.item()}")

        # Additional logging
        if step % 10 == 0:
            print(f"Detailed logging at Epoch: {epoch}, Step: {step}")
            print(f"Input IDs: {batch['input_ids']}")
            print(f"Attention Mask: {batch['attention_mask']}")
            print(f"Training Loss: {loss.item()}")
            wandb.log({"train_loss": loss.item(), "epoch": epoch, "step": step})

    avg_train_loss = train_loss / len(train_data_loader)
    print(f"Epoch {epoch} avg Training Loss of this epoch: {avg_train_loss}")
    print(f"Train dataloader length: {len(train_data_loader)}")
    wandb.log({"avg_train_loss": avg_train_loss, "epoch": epoch})

    # Evaluation
    model.eval()
    eval_loss = 0
    all_preds = []
    all_labels = []
    nsp_true_labels = []
    nsp_all_preds = []
    with torch.no_grad():
        for step, batch in enumerate(eval_data_loader):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            eval_loss += loss.item()

            prediction_logits = outputs.prediction_logits
            seq_relationship_logits = outputs.seq_relationship_logits

            # For MLM
            mlm_preds = torch.argmax(prediction_logits, dim=-1)

            print(f"mlm_preds {mlm_preds}")

            all_preds.extend(mlm_preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

            print(f"all_labels {all_labels}")

            # we still need softmax to convert the logits into probabilities
            # index 0: sequence B is a continuation of sequence A
            # index 1: sequence B is a random sequence

            # For NSP
            nsp_true_labels.extend(batch['next_sentence_label'].cpu().numpy())

            nsp_preds = torch.softmax(seq_relationship_logits, dim=1)
            print(f"seq_relationship_logits: {seq_relationship_logits}")
            print(f"nsp_preds: {nsp_preds}")

            # tensor([[9.9993e-01, 6.7607e-05]], grad_fn=<SoftmaxBackward>)
            # very high value for index 0: high probability of seq_B being a continuation of seq_A
            # which is what we expect

            nsp_preds_labels = torch.argmax(nsp_preds, dim=1)
            nsp_all_preds.extend(nsp_preds_labels.cpu().numpy())

            print(f"nsp_preds_labels: {nsp_preds_labels}")

            print(f"true nsp labels: {nsp_true_labels}")

            print(f"Epoch: {epoch}, Step: {step}, evaluation Loss: {loss.item()}")

            # Additional logging
            if step % 10 == 0:
                print(f"Detailed logging at Epoch: {epoch}, Step: {step}")
                print(f"Input IDs: {batch['input_ids']}")
                print(f"Attention Mask: {batch['attention_mask']}")
                print(f"Evaluation Loss: {loss.item()}")
                wandb.log({"eval_loss": loss.item(), "epoch": epoch, "step": step})

    avg_eval_loss = eval_loss / len(eval_data_loader)
    # print len(eval_data_loader)
    print(f"Eval dataloader length: {len(eval_data_loader)}")
    # def compute_metrics(mlm_preds, mlm_labels, nsp_preds, nsp_labels):
    metrics = compute_metrics(mlm_preds = all_preds, mlm_labels = all_labels, nsp_preds = nsp_all_preds, nsp_labels = nsp_true_labels)
    print(f"Epoch {epoch} avg Evaluation Loss of this epoch: {avg_eval_loss}")
    print(f"Evaluation Metrics: {metrics}")

    # Log evaluation metrics
    wandb.log({"avg eval_loss": avg_eval_loss, "epoch": epoch})
    wandb.log(metrics)

    # Save model checkpoint
    checkpoint_dir = os.path.join(training_args.output_dir, f"checkpoint-epoch-{epoch}")
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.save_pretrained(checkpoint_dir)
    tokenizer.save_pretrained(checkpoint_dir)
    torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))
    torch.save(lr_scheduler.state_dict(), os.path.join(checkpoint_dir, "scheduler.pt"))



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
          5, -100, -100, -100, -100, -100, -100,    8, -100, -100, -100,
         18, -100, -100, -100, -100,   15,   19,    7, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100,   10, -100, -100, -100,    8, -100, -100,   16, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100,   10,    7,   19,
       -100, -100, -100,   10, -100,   14, -100, -100, -100, -100,   13,
         18, -100, -100,    7, -100, -100, -100, -100, -100, -100, -100,
       -100, -100,   24, -100, -100, -100,   13,   17,   11,   20, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100,   11, -100,
         13, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100,   17, -100, -100, -100, -100, -100,   14, -100,    6, -100,
         20, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -1

KeyboardInterrupt: 

FileNotFoundError: [Errno 2] No such file or directory: '/content/small_data_50_epochs_new_metrics_lr2e-6_colab/*'